In [1]:
import tensorflow as tf
from functools import partial
from libs.pipeline_utils import build_sampler_env, sample
from libs.pipeline_utils import wrap_list_with_score, simple_probability_pipeline, probability, meaning_probability
from libs.utils import load_dictionary, load_transformer, top_best, load_dis

C:\Users\mikhail\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [11]:
n_items = 1000
batch_size = 64
enc_seq_len = 64
dec_seq_len = 64

In [3]:
load_dir = "./models/shm_c1"
sampler_args = build_sampler_env(load_dir, batch_size, enc_seq_len, dec_seq_len)

INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519
INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519


In [4]:
dictionary = load_dictionary("./data/")
transformer = load_transformer(load_dir)

In [5]:
with tf.device("/cpu:0"):
    dis_be = load_dis('./models/', 'believability')
    dis_st = load_dis('./models/', 'style')

In [6]:
dis_me = load_dis('./models/', 'meaning')

In [7]:
be_f = partial(probability, model=dis_be, transformer=transformer)
st_f = partial(probability, model=dis_st, transformer=transformer)

In [8]:
sample_f = partial(sample, *sampler_args, 
                   dictionary=dictionary, 
                   transformer=transformer, 
                   n_items=n_items,
                   batch_size=64)

In [9]:
seed_phrase = 'Убей себя, урод'
me_f = partial(meaning_probability, model=dis_me, seed_phrase=seed_phrase)

In [10]:
simple_probability_pipeline(seed_phrase, sample_f, [be_f, st_f, me_f], topn=0.02)

[('боитесь смерть!', 0.035858444777383522),
 ('это он тебе утешил?', 0.033524636000661127)]